In [4]:
import pandas as pd

data = pd.read_excel("Datahub_Agri_Latest.xlsx", sheet_name="Database")

# Unpivot Agri Dataset

In [11]:

import pandas as pd

# Step 1: Read the data
data = pd.read_excel("Datahub_Agri_Latest.xlsx", sheet_name="Database")

# Step 2: Check for a typo and fix the column name if necessary
if 'Latiude' in data.columns:
    data = data.rename(columns={'Latiude': 'Latitude'})  # Fix the typo for Latitude

# Step 3: Define the unit for each indicator
indicator_units = {
    'Area Planted': 'Ha',
    'Area Harvested': 'Ha',
    'Quantity Harvested': 'T',
    'Yield': 'T/Ha',
    'Smallholder Livestock': 'Number',
    'Livestock Farming Industry': 'Number',
    'Total Number of Livestock': 'Number',
    'Quantity': 'Kg',
    'Value': 'USD',
    'Share(%)': '%',
    'Growth(%)': '%',
    'Products': 'N/A',
    'Famers': 'Number',
    'Associates': 'Number',
    'SMEs': 'Number',
    'Company': 'Number'
}

# Step 4: Unpivot the data using pd.melt()
unpivoted_data = data.melt(
    id_vars=[
        'Series Code', 'Series Name', 'Sector', 'Sub-Sector (1)', 'Sub-Sector (2)', 
        'Province', 'Year', 'Source', 'Latitude', 'Longitude', 'Pro code', 
        'Markets'
    ], 
    value_vars=[
        'Area Planted', 'Area Harvested', 'Quantity Harvested', 'Yield', 
        'Smallholder Livestock', 'Livestock Farming Industry', 'Total Number of Livestock', 
        'Livestock Production', 'Famers', 'Associates', 'SMEs', 'Company', 
        'Quantity', 'Value', 'Share(%)', 'Growth(%)'
    ], 
    var_name='Indicator', 
    value_name='Indicator Value'
)

# Step 5: Add the 'Indicator Unit' column using the indicator_units dictionary
unpivoted_data['Indicator Unit'] = unpivoted_data['Indicator'].map(indicator_units)

unpivoted_data['Indicator Value'] = unpivoted_data['Indicator Value'].replace(['', ' ', 'N/A'], pd.NA)
unpivoted_data = unpivoted_data.dropna(subset=['Indicator Value'])

# Optionally, you can save the unpivoted data back to a new Excel file
unpivoted_data.to_excel("Unpivoted_Datahub_Agri_Latest.xlsx", index=False)


# Unpivot Economic Dataset

In [20]:

import pandas as pd

# Step 1: Read the data
data = pd.read_excel("Datahub_Agri_Latest.xlsx", sheet_name="Sheet1")

# Step 2: Check for a typo and fix the column name if necessary
if 'Latiude' in data.columns:
    data = data.rename(columns={'Latiude': 'Latitude'})  # Fix the typo for Latitude

# Step 3: Define the unit for each indicator
indicator_units = {
    'Quantity': 'Kg',
    'Value': 'USD',
    'Share(%)': '%',
    'Growth(%)': '%'
}

# Step 4: Unpivot the data using pd.melt()
unpivoted_data = data.melt(
    id_vars=[
        'Series Code', 'Series Name', 'Sector', 'Sub-Sector (1)', 'Sub-Sector (2)', 
        'Province', 'Year', 'Source', 'Latitude', 'Longitude',
        'Products', 'Markets'
    ], 
    value_vars=[
        'Quantity', 'Value', 'Share(%)', 'Growth(%)'
    ], 
    var_name='Indicator', 
    value_name='Indicator Value'
)

# Step 5: Add the 'Indicator Unit' column using the indicator_units dictionary
unpivoted_data['Indicator Unit'] = unpivoted_data['Indicator'].map(indicator_units)

unpivoted_data['Indicator Value'] = unpivoted_data['Indicator Value'].replace(['', ' ', 'N/A'], pd.NA)
unpivoted_data = unpivoted_data.dropna(subset=['Indicator Value'])

# Step 6: Strip leading and trailing spaces from text columns
text_columns = [
    'Series Name', 'Sector', 'Sub-Sector (1)',
    'Source', 'Products', 'Markets', 'Indicator', 'Indicator Unit'
]

for col in text_columns:
    unpivoted_data[col] = unpivoted_data[col].str.strip()
    
# Optionally, you can save the unpivoted data back to a new Excel file
unpivoted_data.to_excel("Unpivoted_Datahub_Economic.xlsx", index=False)
